In [1]:
%load_ext autoreload
%autoreload 2
from dgnss_utils import panoseti_file_interfaces as pfi
from dgnss_utils import pff
import pandas as pd
from decimal import *
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_dir = '/Users/nico/Downloads/panoseti_test_data/obs_data/data'
run_dir = 'obs_Lick.start_2023-08-29T04:49:58Z.runtype_sci-obs.pffd'

In [3]:
ori = pfi.ObservingRunInterface(data_dir, run_dir)
ori.obs_pff_files

{254: {'img': [],
  'ph': [{'fname': 'start_2023-08-29T04_50_46Z.dp_ph256.bpp_2.module_254.seqno_0.pff',
    'seqno': 0,
    'nframes': 12659,
    'first_unix_t': Decimal('1693284653.770646846'),
    'last_unix_t': Decimal('1693295935.356830584')}]},
 1: {'img': [{'fname': 'start_2023-08-29T04_50_49Z.dp_img16.bpp_2.module_1.seqno_0.pff',
    'seqno': 0,
    'nframes': 1574804,
    'first_unix_t': Decimal('1693284655.214407488'),
    'last_unix_t': Decimal('1693284812.694707488')},
   {'fname': 'start_2023-08-29T04_53_33Z.dp_img16.bpp_2.module_1.seqno_1.pff',
    'seqno': 1,
    'nframes': 1574804,
    'first_unix_t': Decimal('1693284812.694807488'),
    'last_unix_t': Decimal('1693284970.175107488')}],
  'ph': [{'fname': 'start_2023-08-29T04_50_49Z.dp_ph256.bpp_2.module_1.seqno_0.pff',
    'seqno': 0,
    'nframes': 10519,
    'first_unix_t': Decimal('1693284655.602063831'),
    'last_unix_t': Decimal('1693295937.169311272')}]},
 3: {'img': [],
  'ph': [{'fname': 'start_2023-08-29T04_5

In [4]:
fname = ori.obs_pff_files[1]["ph"][0]["fname"]
fpath = f'{data_dir}/{run_dir}/{fname}'

In [ ]:
columns = [
    'timestamp',
    'date',
    'quabo_num',
    'pkt_num',
    'pkt_tai',
    'pkt_nsec',
    'tv_sec',
    'tv_usec',
    # 'img_data'
]
ph_df = pd.DataFrame(columns=columns)

with open(fpath, 'rb') as fp:
    frame_iterator = ori.pulse_height_frame_iterator(fp, 1)
    for j, img in frame_iterator:
        j['timestamp'] = pff.wr_to_unix_decimal(j['pkt_tai'], j['pkt_nsec'], j['tv_sec'])
        # j['img_data'] = img
        ph_df.loc[len(ph_df)] = j
ph_df['date'] = pd.to_datetime(ph_df['timestamp'].astype(np.float64), unit = 's')
ph_df = ph_df.sort_values(by='timestamp')
ph_df

In [ ]:
ph_df['time_differential (ns)'] = ph_df['timestamp'].diff() * 10**9#.dropna()

In [ ]:
plt.figure(figsize=(12,6))
plt.xlabel('Time difference (ns)')
plt.title(f'GNSS timing differential\n{fname}')
plt.axvline(x=ph_df['time_differential (ns)'].mean().astype(np.float64), label='mean', c='r')
plt.axvline(x=ph_df['time_differential (ns)'].median().astype(np.float64), label='median', c='b')

plt.legend()
sns.histplot(ph_df, x='time_differential (ns)', stat='density');

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(
    ph_df,
    x='date',
    y='time_differential (ns)',
    hue='quabo_num',
    size=1.0
)

In [ ]:
plt.figure(figsize=(6, 6))
sns.violinplot(
    ph_df,
    x='quabo_num',
    y='time_differential (ns)',
)